In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# download command to get power cost data from awattar
#!curl "https://api.awattar.de/v1/marketdata?start=1635724800000&end=1667260800000" > awattar_1121_1022.json
#!wget 'http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/data/awattar_1121_1022.json'
#!wget 'http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/data/awattar_0121_1221.json'
#f=open('awattar_1121_1022.json')
f=open('awattar_0121_1221.json') # 2021 data

a=json.load(f)
d=[(x['start_timestamp'],x['marketprice']) for x in a['data']] # convert into 2D list
dfc = pd.DataFrame( d, columns=['timestamp','price'])
# convert epoch ms into datetime and use as index
dfc.index = pd.to_datetime(dfc.timestamp,unit='ms',utc=True)

In [ ]:
dfc

In [ ]:
dfc.describe()

In [ ]:
dfc.loc['2021-12'].price.plot() # plot dec 21

In [ ]:
# resample per week
dfw=dfc['price'].resample('w').mean() 
dfw.plot()
plt.grid()

In [ ]:
# get energy mix data from csv for 2021 & 2022
dfe21=pd.read_csv('http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/data/energy-charts_Oeffentliche_Nettostromerzeugung_in_Deutschland_2021.csv',index_col='Datum (GMT+1)',parse_dates=['Datum (GMT+1)'],engine='python')
dfe22=pd.read_csv('http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/data/energy-charts_Oeffentliche_Nettostromerzeugung_in_Deutschland_2022.csv',index_col='Datum (GMT+1)',parse_dates=['Datum (GMT+1)'],engine='python')


In [ ]:
dfe=pd.concat((dfe21,dfe22))

In [ ]:
dfe.describe()

In [ ]:
dfe.Solar.plot();

In [ ]:
dfe['total']=dfe.sum(axis=1)-dfe.Last

In [ ]:
# CO2 Emission g/kWh-Strom (aus Tab 3 https://www.umweltbundesamt.de/sites/default/files/medien/1410/publikationen/2022-04-13_cc_15-2022_strommix_2022_fin_bf.pdf)
co2fact={"Braunkohle":1146,"Steinkohle":852,"Erdgas":397}
co2fact['Oel']=500 # GD estimate

In [ ]:
dfe['co2frac'] = (co2fact['Steinkohle']*dfe.Steinkohle+co2fact['Braunkohle']*dfe.Braunkohle+
                  co2fact['Erdgas']*dfe.Erdgas+co2fact['Oel']*dfe['Öl'])/dfe['total']

In [ ]:
dfe

In [ ]:
co2fact

In [ ]:
# data in 15 min intervals --> avg per hour
dfeh=dfe.resample('1H').mean()


In [ ]:
# select data from 2021
#dfeh2= dfeh['2021-11-01':'2022-10-31']
dfeh2= dfeh['2021-01-01':'2021-12-31']


In [ ]:
# combine energy mix and cost df
dfehc=dfeh2.join(dfc)

In [ ]:
dfehc.describe()

In [ ]:
dfehc[dfehc.co2frac>600]

In [ ]:
fig = plt.figure(1, figsize=(14,6))
ax1 = fig.add_subplot(121)
dfehc.hist('co2frac',bins=50,ax=ax1)
ax2 = fig.add_subplot(122)
dfehc.hist('price',bins=50,ax=ax2);

In [ ]:
dfehc.plot.scatter('price','co2frac',marker='.')
plt.grid()

In [ ]:
import seaborn as sns
sns.jointplot(data=dfehc,x='price',y='co2frac', kind='hist');


### Calculate cpu loss, cost and co2 savings

Global ansatz: use 1 year data to determine percentiles for thresholds

In [ ]:
df = dfehc
# calculate cost & co2 saving and cpu loss
l=np.median(df.price) # median price
h=np.max(df.price)    # maximum price
print(f"Median price={l:.2f}  Max price={h:.2f}")
N=50
pcs=np.linspace(0.,100.,N)
tl = np.percentile(df.price,pcs) # get percentiles



nt = df.price.count()
st = df.price.sum()
co2t = df.co2frac.sum()
nva = np.zeros(N,dtype=float)
sva = np.zeros(N,dtype=float)
cva = np.zeros(N,dtype=float)

# simple algorithm: take global threshold values between 0 and max-cost
for i,x in enumerate(tl[::-1]):
    nv = df.price[df.price>x].count() # count entries larger cost x
    sv = df.price[df.price>x].sum()   # sum entries larger cost x
    cv = df.co2frac[df.price>x].sum()
    nva[i] = nv
    sva[i] = sv
    cva[i] = cv
    
    
sfact = 0.4 # cpu-cycle/power-factor during reducedoperation
losscpu = (1-sfact)*nva/nt # normalized cpu lost
losscost = (1-sfact)*sva/st # normalized cost reduction
lossco2  = (1-sfact)*cva/co2t

In [ ]:
for i in range(0,len(losscpu),2):
    print(f"{losscpu[i]:.5f}  {losscost[i]:.5f}  {lossco2[i]:.5f}")

In [ ]:
#
plt.figure(figsize=(12,8))
plt.plot(losscpu,losscost,label='cost')
plt.plot(losscpu,lossco2,label='co2')
plt.plot(losscpu,losscpu)
plt.grid()
plt.xlabel('CPU loss')
plt.ylabel('Cost saving')
plt.legend()

### Calculate cpu loss, cost and co2 savings

local ansatz: use last weeks data to determine percentiles for thresholds, update daily

In [ ]:
df = dfehc
deltad=24
deltaw=7*24
nt=len(df)

N=50
nva = np.zeros(N,dtype=float)
sva = np.zeros(N,dtype=float)
cva = np.zeros(N,dtype=float)

# recalculate thresholds daily using last week avg
ilist = list(range(deltaw,nt,deltad))
pcs=np.linspace(0.,100.,N)
for ind in ilist:
    #print(ind)
    # determine percentiles last 7 days
    dfl = df.iloc[ind-deltaw:ind]
    tl = np.percentile(dfl.price,pcs)
    # apply to next 24h
    dfd = df.iloc[ind:ind+deltad]
    for i,x in enumerate(tl[::-1]):
        nv = dfd.price[dfd.price>x].count() # count entries larger cost x
        sv = dfd.price[dfd.price>x].sum()   # sum entries larger cost x
        cv = dfd.co2frac[dfd.price>x].sum()
        nva[i] += nv
        sva[i] += sv
        cva[i] += cv

dfl = df.iloc[deltaw:]
nt = dfl.price.count()
st = dfl.price.sum()
co2t = dfl.co2frac.sum()



    
sfact = 0.4 # cpu-cycle/power-factor during reducedoperation
losscpu = (1-sfact)*nva/nt # normalized cpu lost
losscost = (1-sfact)*sva/st # normalized cost reduction
lossco2  = (1-sfact)*cva/co2t

In [ ]:
for i in range(0,len(losscpu),2):
    print(f"{losscpu[i]:.5f}  {losscost[i]:.5f}  {lossco2[i]:.5f}")

In [ ]:
#
plt.figure(figsize=(12,8))
plt.plot(losscpu,losscost,label='cost')
plt.plot(losscpu,lossco2,label='co2')
plt.plot(losscpu,losscpu)
plt.grid()
plt.xlabel('CPU loss')
plt.ylabel('Cost saving')
plt.legend()